In [168]:
#notebook for perform ml algorithms
#import necessary packages
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import seaborn as sn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
#import and prepare data
player_ratings_19 = pd.read_csv('Data/2019-20/Players/player_ratings_19_a_09.csv')
features_19 = pd.read_csv('Data/2019-20/Fixtures/PL19_20_features.csv')
#exclude gameweeks 1-3
features_19.drop(features_19[features_19.Wk_h < 4].index, inplace=True)
player_ratings_20 = pd.read_csv('Data/2020-21/Players/player_ratings_a_09.csv')
features_20 = pd.read_csv('Data/2020-21/Fixtures/PL20_21_features.csv')
features_20.drop(features_20[features_20.Wk_h < 4].index, inplace=True)
player_ratings_21 = pd.read_csv('Data/2021-22/Players/player_ratings_21_a_09.csv')
features_21 = pd.read_csv('Data/2021-22/Fixtures/PL21_22_features.csv')
features_21.drop(features_21[features_21.Wk_h < 4].index, inplace=True)
player_ratings_20.replace(['Brighton & Hove Albion','Manchester United', 'Newcastle United', 'Sheffield United', 'Tottenham Hotspur', 'West Bromwich Albion', 'West Ham United','Wolverhampton Wanderers'], ['Brighton', 'Manchester Utd','Newcastle Utd', 'Sheffield Utd', 'Tottenham','West Brom' ,'West Ham', 'Wolves'], inplace=True)
player_ratings_20['game_home_ID'] = player_ratings_20['HomeTeam'] + player_ratings_20['Date']
player_ratings_20['game_away_ID'] = player_ratings_20['AwayTeam'] + player_ratings_20['Date']
player_ratings_20.drop(player_ratings_20[player_ratings_20.Wk_h < 4].index, inplace=True)
features_21

,Unnamed: 0,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,Elo_h_after,Elo_a_after,Elo_h_before,Elo_a_before,goalsDifference,shotsDifference,cornersDifference,ewmDifference,elo_difference,form_difference
30,30,E0,2021-09-11,12:30,Crystal Palace,Tottenham,3,0,H,0,...,1505.52,1525.14,1488.18,1542.48,-1.500000,-4.000000,-1.500000,-6.783784,-54.30,-4.000000
31,31,E0,2021-09-11,15:00,Arsenal,Norwich City,1,0,H,0,...,1472.46,1441.35,1457.52,1456.29,0.000000,2.500000,6.000000,0.000000,1.23,0.000000
32,32,E0,2021-09-11,15:00,Brentford,Brighton,0,1,A,0,...,1498.17,1529.25,1513.11,1514.31,-1.000000,-5.000000,-5.000000,-1.081081,-1.20,-1.000000
33,33,E0,2021-09-11,15:00,Leicester City,Manchester City,0,1,A,0,...,1498.71,1528.77,1513.71,1513.77,-1.000000,-0.500000,-9.000000,0.027027,-0.06,1.500000
34,34,E0,2021-09-11,15:00,Manchester Utd,Newcastle Utd,4,1,H,1,...,1539.48,1458.09,1526.88,1470.69,3.500000,6.500000,-0.500000,5.792793,56.19,3.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,E0,2022-05-22,16:00,Crystal Palace,Manchester Utd,1,0,H,1,...,1498.11,1506.66,1481.31,1523.46,-1.333333,-1.666667,1.166667,-12.982307,-42.15,0.000000
376,376,E0,2022-05-22,16:00,Leicester City,Southampton,4,1,H,0,...,1503.48,1420.47,1490.97,1432.98,1.833333,4.666667,-2.166667,8.867102,57.99,0.333333
377,377,E0,2022-05-22,16:00,Liverpool,Wolves,3,1,H,1,...,1716.99,1461.21,1711.08,1467.12,5.000000,6.166667,10.000000,37.769667,243.96,4.500000
378,378,E0,2022-05-22,16:00,Manchester City,Aston Villa,3,2,H,0,...,1703.55,1460.85,1697.25,1467.15,9.000000,1.333333,4.500000,44.976998,230.10,4.000000


In [4]:
#merge dataframes of 3 seasons
PL_2019_total =  pd.merge(features_19, player_ratings_19, on= 'game_home_ID', how='left')
PL_2020_total =  pd.merge(features_20, player_ratings_20, on= 'game_home_ID', how='left')
PL_2021_total =  pd.merge(features_21, player_ratings_21, on= 'game_home_ID', how='left')
frames = [PL_2019_total, PL_2020_total, PL_2021_total]
PL_19_20_21_all_features = pd.concat(frames)

,Unnamed: 0_x,Div,Date_x,Time_x,HomeTeam_x,AwayTeam_x,FTHG_x,FTAG_x,FTR_x,HTHG,...,rating_a_gk,rating_a_df,rating_a_mf,rating_a_att,rating_a_bench,influence_a,creativity_a,threat_a,bps_a,selected_a
0,30,E0,2019-08-31,12:30,Southampton,Manchester Utd,1,1,D,0,...,20.641209,180.145904,56.972852,48.633782,16.068837,119.153153,168.309910,178.081081,94.243243,7.569864e+06
1,31,E0,2019-08-31,15:00,Chelsea,Sheffield Utd,2,2,D,2,...,23.885922,130.639893,61.162521,34.465713,5.870770,171.733333,113.054955,109.324324,142.297297,1.280639e+06
2,32,E0,2019-08-31,15:00,Crystal Palace,Aston Villa,1,0,H,0,...,29.488526,64.773615,82.120631,79.960598,0.916340,195.585586,103.043243,81.135135,196.090090,1.682107e+06
3,33,E0,2019-08-31,15:00,Leicester City,Bournemouth,3,1,H,2,...,31.163130,58.769298,82.006199,40.354356,23.149859,195.951351,80.441441,122.513514,126.126126,3.708388e+06
4,34,E0,2019-08-31,15:00,Manchester City,Brighton,4,0,H,2,...,32.895815,81.276387,25.286296,67.316595,17.219749,132.708108,133.936937,152.936937,108.342342,2.252684e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,375,E0,2022-05-22,16:00,Crystal Palace,Manchester Utd,1,0,H,1,...,11.750857,96.896682,69.117198,19.445436,47.452827,134.023334,103.132912,78.215625,112.788455,3.120155e+06
346,376,E0,2022-05-22,16:00,Leicester City,Southampton,4,1,H,0,...,30.426248,90.718082,38.302624,27.739725,21.677295,184.049913,35.385130,50.186897,128.755329,9.541940e+05
347,377,E0,2022-05-22,16:00,Liverpool,Wolves,3,1,H,1,...,0.634142,158.514461,67.462373,28.278849,69.217545,154.614288,170.164161,153.131430,132.815157,2.843078e+06
348,378,E0,2022-05-22,16:00,Manchester City,Aston Villa,3,2,H,0,...,0.000000,138.256940,193.946287,15.334034,27.109975,202.612485,230.250828,198.143528,148.016855,5.887781e+06


In [5]:
#calculate differences for player ratings
PL_19_20_21_all_features['gk_difference'] = PL_19_20_21_all_features['rating_h_gk'] - PL_19_20_21_all_features['rating_a_gk']
PL_19_20_21_all_features['df_difference'] = PL_19_20_21_all_features['rating_h_df'] - PL_19_20_21_all_features['rating_a_df']
PL_19_20_21_all_features['mf_difference'] = PL_19_20_21_all_features['rating_h_mf'] - PL_19_20_21_all_features['rating_a_mf']
PL_19_20_21_all_features['att_difference'] = PL_19_20_21_all_features['rating_h_att'] - PL_19_20_21_all_features['rating_a_att']
PL_19_20_21_all_features['bench_difference'] = PL_19_20_21_all_features['rating_h_bench'] - PL_19_20_21_all_features['rating_a_bench']
PL_19_20_21_all_features['influence_difference'] = PL_19_20_21_all_features['influence_h'] - PL_19_20_21_all_features['influence_a']
PL_19_20_21_all_features['creativity_difference'] = PL_19_20_21_all_features['creativity_h'] - PL_19_20_21_all_features['creativity_a']
PL_19_20_21_all_features['threat_difference'] = PL_19_20_21_all_features['threat_h'] - PL_19_20_21_all_features['threat_a']
PL_19_20_21_all_features['bps_difference'] = PL_19_20_21_all_features['bps_h'] - PL_19_20_21_all_features['bps_a']
PL_19_20_21_all_features['selected_difference'] = PL_19_20_21_all_features['selected_h'] - PL_19_20_21_all_features['selected_a']
PL_19_20_21_all_features['overall_h'] = PL_19_20_21_all_features['rating_h_gk'] + PL_19_20_21_all_features['rating_h_df'] + PL_19_20_21_all_features['rating_h_mf'] + PL_19_20_21_all_features['rating_h_att'] + PL_19_20_21_all_features['rating_h_bench']
PL_19_20_21_all_features['overall_a'] = PL_19_20_21_all_features['rating_a_gk'] + PL_19_20_21_all_features['rating_a_df'] + PL_19_20_21_all_features['rating_a_mf'] + PL_19_20_21_all_features['rating_a_att'] + PL_19_20_21_all_features['rating_a_bench']
PL_19_20_21_all_features['overall_difference'] = PL_19_20_21_all_features['overall_h'] - PL_19_20_21_all_features['overall_a']
PL_19_20_21_all_features

,Unnamed: 0_x,Div,Date_x,Time_x,HomeTeam_x,AwayTeam_x,FTHG_x,FTAG_x,FTR_x,HTHG,...,att_difference,bench_difference,influence_difference,creativity_difference,threat_difference,bps_difference,selected_difference,overall_h,overall_a,overall_difference
0,30,E0,2019-08-31,12:30,Southampton,Manchester Utd,1,1,D,0,...,-8.696108,-13.029503,29.643243,-56.994595,-91.072072,72.252252,-7.090981e+06,237.478074,322.462585,-84.984511
1,31,E0,2019-08-31,15:00,Chelsea,Sheffield Utd,2,2,D,2,...,97.174211,2.612414,128.670270,128.120721,139.981982,105.180180,1.377962e+06,419.652975,256.024819,163.628156
2,32,E0,2019-08-31,15:00,Crystal Palace,Aston Villa,1,0,H,0,...,-22.700878,11.903258,8.221622,-70.580180,21.954955,-72.369369,2.088097e+06,211.433752,257.259711,-45.825959
3,33,E0,2019-08-31,15:00,Leicester City,Bournemouth,3,1,H,2,...,-8.118451,15.382999,6.879279,12.309009,-11.549550,15.963964,-1.301976e+06,303.887309,235.442842,68.444467
4,34,E0,2019-08-31,15:00,Manchester City,Brighton,4,0,H,2,...,20.274545,16.194642,169.688288,98.920721,114.045045,114.009009,7.638306e+06,475.117929,223.994842,251.123087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,375,E0,2022-05-22,16:00,Crystal Palace,Manchester Utd,1,0,H,1,...,-2.441324,83.527907,-55.839647,-71.843929,-27.652552,-44.959147,6.086770e+05,267.620547,244.663001,22.957546
346,376,E0,2022-05-22,16:00,Leicester City,Southampton,4,1,H,0,...,29.145907,22.307791,24.274186,8.288232,-30.099457,10.599418,4.572563e+06,279.762870,208.863974,70.898896
347,377,E0,2022-05-22,16:00,Liverpool,Wolves,3,1,H,1,...,-1.378848,189.332683,-37.028619,-137.912185,-66.928122,-36.540463,6.056832e+06,443.826626,324.107370,119.719256
348,378,E0,2022-05-22,16:00,Manchester City,Aston Villa,3,2,H,0,...,25.620656,77.550764,-56.131097,-25.603259,48.868307,-16.173457,4.954865e+06,483.907694,374.647236,109.260458


In [151]:
#prepare Data for ML Algos, including all features from benchmark + rating differences
#add column for H A X
conditions = [
    (PL_19_20_21_all_features['FTR_x'] == 'H'),
    (PL_19_20_21_all_features['FTR_x'] == 'A'),
    (PL_19_20_21_all_features['FTR_x'] == 'D')]
choices = [0, 1, 2]
PL_19_20_21_all_features['identifier'] = np.select(conditions, choices)
y = PL_19_20_21_all_features['identifier'].to_numpy()
#all features used for benchmark
feature_names_benchmark = ['Rating difference', 'Rating_Home', 'Rating_Away', 'ewmH', 'ewmA', 'ewmDifference', 'Elo_h_before','Elo_a_before', 'elo_difference', 'goalsH', 'goalsA', 'goalsDifference', 'shotsH', 'shotsA','shotsDifference', 'cornersH', 'cornersA', 'cornersDifference', 'formH', 'formA', 'form_difference']
#features including player stats
feature_names_player_stats = ['rating_h_gk', 'Rating difference', 'Rating_Home', 'Rating_Away', 'ewmH', 'ewmA', 'ewmDifference', 'Elo_h_before','Elo_a_before', 'elo_difference', 'goalsH', 'goalsA', 'goalsDifference', 'shotsH', 'shotsA','shotsDifference', 'cornersH', 'cornersA', 'cornersDifference', 'formH', 'formA', 'form_difference', 'rating_a_gk', 'rating_h_df', 'rating_a_df', 'rating_h_mf', 'rating_a_mf', 'rating_h_att', 'rating_a_att', 'rating_h_bench', 'rating_a_bench', 'influence_h', 'influence_a', 'creativity_h', 'creativity_a', 'threat_h', 'threat_a', 'bps_h', 'bps_a', 'selected_h', 'selected_a', 'gk_difference', 'df_difference', 'mf_difference', 'att_difference', 'bench_difference','overall_difference' ,'influence_difference', 'creativity_difference', 'threat_difference', 'bps_difference', 'selected_difference', 'overall_h', 'overall_a']
PL_19_20_21_all_features

,Unnamed: 0_x,Div,Date_x,Time_x,HomeTeam_x,AwayTeam_x,FTHG_x,FTAG_x,FTR_x,HTHG,...,bench_difference,influence_difference,creativity_difference,threat_difference,bps_difference,selected_difference,overall_h,overall_a,overall_difference,identifier
0,30,E0,2019-08-31,12:30,Southampton,Manchester Utd,1,1,D,0,...,-13.029503,29.643243,-56.994595,-91.072072,72.252252,-7.090981e+06,237.478074,322.462585,-84.984511,2
1,31,E0,2019-08-31,15:00,Chelsea,Sheffield Utd,2,2,D,2,...,2.612414,128.670270,128.120721,139.981982,105.180180,1.377962e+06,419.652975,256.024819,163.628156,2
2,32,E0,2019-08-31,15:00,Crystal Palace,Aston Villa,1,0,H,0,...,11.903258,8.221622,-70.580180,21.954955,-72.369369,2.088097e+06,211.433752,257.259711,-45.825959,0
3,33,E0,2019-08-31,15:00,Leicester City,Bournemouth,3,1,H,2,...,15.382999,6.879279,12.309009,-11.549550,15.963964,-1.301976e+06,303.887309,235.442842,68.444467,0
4,34,E0,2019-08-31,15:00,Manchester City,Brighton,4,0,H,2,...,16.194642,169.688288,98.920721,114.045045,114.009009,7.638306e+06,475.117929,223.994842,251.123087,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,375,E0,2022-05-22,16:00,Crystal Palace,Manchester Utd,1,0,H,1,...,83.527907,-55.839647,-71.843929,-27.652552,-44.959147,6.086770e+05,267.620547,244.663001,22.957546,0
346,376,E0,2022-05-22,16:00,Leicester City,Southampton,4,1,H,0,...,22.307791,24.274186,8.288232,-30.099457,10.599418,4.572563e+06,279.762870,208.863974,70.898896,0
347,377,E0,2022-05-22,16:00,Liverpool,Wolves,3,1,H,1,...,189.332683,-37.028619,-137.912185,-66.928122,-36.540463,6.056832e+06,443.826626,324.107370,119.719256,0
348,378,E0,2022-05-22,16:00,Manchester City,Aston Villa,3,2,H,0,...,77.550764,-56.131097,-25.603259,48.868307,-16.173457,4.954865e+06,483.907694,374.647236,109.260458,0


In [89]:
#split Data in training and test set and train model
#training set = season 19/20 and 20/21
#test set = season 21/22

#all features from benchmark
X_benchmark = PL_19_20_21_all_features[feature_names_benchmark].to_numpy()
#all features including player stats
X_player_stats = PL_19_20_21_all_features[feature_names_player_stats].to_numpy()
X_train_player_stats = X_player_stats[:750]
X_test_player_stats = X_player_stats[-350:]
X_train_benchmark = X_benchmark[:750]
X_test_benchmark = X_benchmark[-350:]
y_train_player_stats = y[:750]
y_test_player_stats = y[-350:]

In [91]:
#Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [102]:
#set up Random forest classifier
rand_clf = RandomForestClassifier(random_state=6)
rand_clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 6,
 'verbose': 0,
 'warm_start': False}

In [93]:
#train model with training data
rand_clf.fit(X_train_player_stats, y_train_player_stats)


RandomForestClassifier(random_state=6)

In [56]:
#perform random search for hyperparameter tuning
rand_search = RandomizedSearchCV(estimator = rand_clf, param_distributions = random_grid, n_iter = 150, cv = 4, verbose=3, random_state=42, n_jobs = -1)
rand_search.fit(X_train_player_stats, y_train_player_stats)

In [58]:
#display best parameters
rand_search.best_params_

{'n_estimators': 1000,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 80,
 'bootstrap': True}

In [94]:
#set up model with best parameters from random search
rand_clf_res = RandomForestClassifier(
 n_estimators = 1000,
 min_samples_split = 10,
 min_samples_leaf = 4,
 max_features = 'auto',
 max_depth = 80)

In [77]:
#create grid close to values from random search
grid_param = {
 'n_estimators': [800, 1000, 1200, 2000],
 'min_samples_split': [5,10,15],
 'min_samples_leaf':  [2,4,6],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [60, 80, 100]
}

In [78]:
#perform grid search
grid_search = GridSearchCV(estimator=rand_clf, param_grid=grid_param, cv=5, n_jobs=-1, verbose=3)
grid_search.fit(X_train_player_stats, y_train_player_stats)

In [80]:
#display best parameters
grid_search.best_params_

{'max_depth': 60,
 'max_features': 'auto',
 'min_samples_leaf': 6,
 'min_samples_split': 15,
 'n_estimators': 2000}

In [95]:
#build model with best parameters from grid search
grid_clf = RandomForestClassifier(
 n_estimators = 800,
 min_samples_split = 3,
 min_samples_leaf = 2,
 max_features = 'auto',
 max_depth = 80)

In [97]:
def find_best_model(clf):
    avg_score = []
    highscore = 0
    out = 0
    for i in range (0,10):
        #model = clf.fit(X_train_benchmark, y_train_player_stats)
        #score = model.score(X_test_benchmark, y_test_player_stats)
        model = clf.fit(X_train_player_stats, y_train_player_stats)
        score = model.score(X_test_player_stats, y_test_player_stats)
        avg_score.append(score)
        #print('Score', score)
        #if score >= 0.65:
         #   return model
    return avg_score


test = find_best_model(rand_clf)
test2 = find_best_model(rand_clf_res)
test3 = find_best_model(grid_clf)
print('rand_clf_mean:', np.mean(test))
print('rand_clf_max:', np.max(test))
print('rand_clf_res_mean:', np.mean(test2))
print('rand_clf_res_max:', np.max(test2))
print('grid_clf_mean:', np.mean(test3))
print('grid_clf_max:', np.max(test3))

rand_clf_mean: 0.5942857142857143
rand_clf_max: 0.5942857142857143
rand_clf_res_mean: 0.5951428571428572
rand_clf_res_max: 0.6028571428571429
grid_clf_mean: 0.5977142857142858
grid_clf_max: 0.6114285714285714


In [121]:
def find_best_model(clf):
    avg_score = []
    highscore = 0
    out = 0
    for i in range (0,200):
        #model = clf.fit(X_train_benchmark, y_train_player_stats)
        #score = model.score(X_test_benchmark, y_test_player_stats)
        model = clf.fit(X_train_player_stats, y_train_player_stats)
        score = model.score(X_test_player_stats, y_test_player_stats)
        #avg_score.append(score)
        print('Score', score)
        if score > 0.61:
            return model
    return model
#rand_clf = find_best_model(rand_clf)
#rand_clf_res = find_best_model(rand_clf_res)
#grid_clf = find_best_model(grid_clf)

Score 0.5971428571428572
Score 0.6
Score 0.6028571428571429
Score 0.6
Score 0.6085714285714285
Score 0.5914285714285714
Score 0.6085714285714285
Score 0.5971428571428572
Score 0.6028571428571429
Score 0.5914285714285714
Score 0.6
Score 0.6057142857142858
Score 0.5857142857142857
Score 0.5971428571428572
Score 0.5885714285714285
Score 0.6
Score 0.6057142857142858
Score 0.5942857142857143
Score 0.5971428571428572
Score 0.5942857142857143
Score 0.5914285714285714
Score 0.6
Score 0.5914285714285714
Score 0.5971428571428572
Score 0.5942857142857143
Score 0.6
Score 0.6028571428571429
Score 0.6028571428571429
Score 0.6
Score 0.5971428571428572
Score 0.5914285714285714
Score 0.6028571428571429
Score 0.5971428571428572
Score 0.6
Score 0.5914285714285714
Score 0.5971428571428572
Score 0.6028571428571429
Score 0.6028571428571429
Score 0.6028571428571429
Score 0.5914285714285714
Score 0.6
Score 0.5942857142857143
Score 0.6
Score 0.6057142857142858
Score 0.5914285714285714
Score 0.6057142857142858


In [152]:
print('Rand_clf', rand_clf.score(X_test_player_stats, y_test_player_stats))
print('Rand_clf_res',rand_clf_res.score(X_test_player_stats, y_test_player_stats))
print('Grid_clf_res',grid_clf.score(X_test_player_stats, y_test_player_stats))


Rand_clf 0.5942857142857143
Rand_clf_res 0.5971428571428572
Grid_clf_res 0.6114285714285714


In [123]:
rf_model_player_stats = grid_clf
rf_model_benchmark = RandomForestClassifier(
 n_estimators = 800,
 min_samples_split = 3,
 min_samples_leaf = 2,
 max_features = 'auto',
 max_depth = 80)
rf_model_benchmark = rf_model_benchmark.fit(X_train_benchmark, y_train_player_stats)

In [153]:
#rf_model_player_stats = grid_clf
print('rf_model_benchmark',rf_model_benchmark.score(X_test_benchmark, y_test_player_stats))
print('rf_model_player_stats',rf_model_player_stats.score(X_test_player_stats, y_test_player_stats))

rf_model_benchmark 0.5857142857142857
rf_model_player_stats 0.6114285714285714


In [125]:
probs_benchmark = rf_model_benchmark.predict_proba(X=X_test_benchmark)
probs_benchmark

array([[0.53359623, 0.26078274, 0.20562103],
       [0.71834524, 0.09598512, 0.18566964],
       [0.15145685, 0.61402827, 0.23451488],
       ...,
       [0.77162351, 0.12195387, 0.10642262],
       [0.76151339, 0.12294494, 0.11554167],
       [0.22986756, 0.54137946, 0.22875298]])

In [156]:
votes_benchmark = rf_model_benchmark.predict(X=X_test_benchmark)
votes_player_stats = rf_model_player_stats.predict(X=X_test_player_stats)
votes_benchmark = pd.DataFrame(test, columns=['benchmark_vote'])
votes_player_stats = pd.DataFrame(test, columns=['player_stats_vote'])
votes_player_stats

,player_stats_vote
0,0
1,0
2,1
3,1
4,0
...,...
345,2
346,0
347,0
348,0


In [126]:
probs_player_stats = rf_model_player_stats.predict_proba(X=X_test_player_stats)
probs_player_stats

array([[0.63102183, 0.20013988, 0.16883829],
       [0.66064435, 0.13628125, 0.2030744 ],
       [0.15189881, 0.674     , 0.17410119],
       ...,
       [0.60490228, 0.26406994, 0.13102778],
       [0.62922321, 0.20953571, 0.16124107],
       [0.17903423, 0.5919003 , 0.22906548]])

In [129]:
probs_benchmark = pd.DataFrame(probs_benchmark, columns=['H', 'D', 'A'])
probs_benchmark.to_csv('Data/Total/probs_benchmark.csv')
probs_benchmark

,H,D,A
0,0.533596,0.260783,0.205621
1,0.718345,0.095985,0.185670
2,0.151457,0.614028,0.234515
3,0.100253,0.720705,0.179042
4,0.850411,0.069783,0.079807
...,...,...,...
345,0.204146,0.318307,0.477548
346,0.547199,0.146458,0.306342
347,0.771624,0.121954,0.106423
348,0.761513,0.122945,0.115542


In [128]:
probs_player_stats = pd.DataFrame(probs_player_stats, columns=['H', 'D', 'A'])
probs_player_stats.to_csv('Data/Total/probs_player_stats.csv')
probs_player_stats

,H,D,A
0,0.631022,0.200140,0.168838
1,0.660644,0.136281,0.203074
2,0.151899,0.674000,0.174101
3,0.112982,0.771997,0.115021
4,0.848845,0.075893,0.075262
...,...,...,...
345,0.259396,0.378656,0.361948
346,0.626677,0.173323,0.200000
347,0.604902,0.264070,0.131028
348,0.629223,0.209536,0.161241


In [159]:
probs_player_stats_tmp = PL_19_20_21_all_features[-350:].join(probs_player_stats)
probs_player_stats_tmp = probs_player_stats_tmp[['HomeTeam_x', 'AwayTeam_x', 'FTR_x', 'H', 'D', 'A']]
probs_player_stats_tmp = probs_player_stats_tmp.rename(columns={'HomeTeam_x':'HomeTeam','AwayTeam_x':'AwayTeam', 'FTR_x': 'FTR'})
probs_player_stats_tmp = probs_player_stats_tmp[-350:].join(votes_player_stats)

conditions = [
    (probs_player_stats_tmp['FTR'] == 'H'),
    (probs_player_stats_tmp['FTR'] == 'A'),
    (probs_player_stats_tmp['FTR'] == 'D')]
choices = [0, 1, 2]
probs_player_stats_tmp['result'] = np.select(conditions, choices)

probs_player_stats_tmp


,HomeTeam,AwayTeam,FTR,H,D,A,player_stats_vote,result
0,Crystal Palace,Tottenham,H,0.631022,0.200140,0.168838,0,0
1,Arsenal,Norwich City,H,0.660644,0.136281,0.203074,0,0
2,Brentford,Brighton,A,0.151899,0.674000,0.174101,1,1
3,Leicester City,Manchester City,A,0.112982,0.771997,0.115021,1,1
4,Manchester Utd,Newcastle Utd,H,0.848845,0.075893,0.075262,0,0
...,...,...,...,...,...,...,...,...
345,Crystal Palace,Manchester Utd,H,0.259396,0.378656,0.361948,2,0
346,Leicester City,Southampton,H,0.626677,0.173323,0.200000,0,0
347,Liverpool,Wolves,H,0.604902,0.264070,0.131028,0,0
348,Manchester City,Aston Villa,H,0.629223,0.209536,0.161241,0,0


In [160]:
probs_player_stats_tmp.to_csv('Data/Total/probs_player_stats_info.csv')

probs_player_stats_tmp

,HomeTeam,AwayTeam,FTR,H,D,A,player_stats_vote,result
0,Crystal Palace,Tottenham,H,0.631022,0.200140,0.168838,0,0
1,Arsenal,Norwich City,H,0.660644,0.136281,0.203074,0,0
2,Brentford,Brighton,A,0.151899,0.674000,0.174101,1,1
3,Leicester City,Manchester City,A,0.112982,0.771997,0.115021,1,1
4,Manchester Utd,Newcastle Utd,H,0.848845,0.075893,0.075262,0,0
...,...,...,...,...,...,...,...,...
345,Crystal Palace,Manchester Utd,H,0.259396,0.378656,0.361948,2,0
346,Leicester City,Southampton,H,0.626677,0.173323,0.200000,0,0
347,Liverpool,Wolves,H,0.604902,0.264070,0.131028,0,0
348,Manchester City,Aston Villa,H,0.629223,0.209536,0.161241,0,0


In [217]:
y_test_player_stats_change = y_test_player_stats
votes_player_stats_change = votes_player_stats
votes_player_stats_change = votes_player_stats_change.to_numpy()
y_test_player_stats_change.shape
votes_player_stats_change = votes_player_stats_change[:,0]

In [218]:
my_dict = {0: 'H', 1: 'A', 2: 'D'}
#my_dict
y_test_player_stats_change = [my_dict[zi] for zi in y_test_player_stats_change]
votes_player_stats_change = [my_dict[zi] for zi in votes_player_stats_change]
votes_player_stats_change

['H',
 'H',
 'A',
 'A',
 'H',
 'D',
 'A',
 'H',
 'A',
 'H',
 'D',
 'A',
 'A',
 'H',
 'D',
 'A',
 'H',
 'H',
 'A',
 'A',
 'A',
 'A',
 'H',
 'A',
 'D',
 'D',
 'D',
 'A',
 'H',
 'D',
 'D',
 'D',
 'H',
 'H',
 'H',
 'D',
 'D',
 'H',
 'A',
 'D',
 'A',
 'A',
 'H',
 'H',
 'D',
 'H',
 'A',
 'A',
 'A',
 'D',
 'A',
 'H',
 'H',
 'H',
 'D',
 'H',
 'A',
 'D',
 'A',
 'H',
 'H',
 'D',
 'H',
 'H',
 'A',
 'H',
 'A',
 'A',
 'D',
 'A',
 'H',
 'D',
 'H',
 'H',
 'A',
 'H',
 'H',
 'A',
 'A',
 'A',
 'A',
 'D',
 'D',
 'H',
 'A',
 'A',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'A',
 'H',
 'A',
 'H',
 'H',
 'H',
 'H',
 'A',
 'A',
 'A',
 'H',
 'H',
 'A',
 'A',
 'H',
 'H',
 'A',
 'H',
 'H',
 'A',
 'A',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'A',
 'A',
 'H',
 'A',
 'A',
 'H',
 'H',
 'H',
 'A',
 'H',
 'H',
 'D',
 'A',
 'A',
 'A',
 'H',
 'A',
 'H',
 'H',
 'A',
 'H',
 'D',
 'H',
 'D',
 'D',
 'A',
 'H',
 'A',
 'H',
 'A',
 'A',
 'D',
 'A',
 'H',
 'H',
 'A',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'A'

In [ ]:
cm= confusion_matrix(y_test_player_stats_change,votes_player_stats_change, labels=rf_model_player_stats.classes_)
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True,cmap="Blues", fmt="d")

#disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf_model_player_stats.classes_)
#disp.plot()
#plt.show()


#print(classification_report(y_test_player_stats,votes_player_stats))
#print(accuracy_score(y_test_player_stats,votes_player_stats))

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x000001A742813520> (for post_execute):


KeyboardInterrupt: 